In [1]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic

In [2]:
sizes = [10,20,50,100,200,500,1000]
density = [0.2,0.5,0.8,1.0]
noise_levels = [0.0, 0.1, 0.5, 0.8]
negative_values = [True, False] 

In [3]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [9]:
problem = create_problem(1000, density=0.15, noise_level=10, negative_values=False)

In [ ]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
import matplotlib.pyplot as plt
nx.draw_circular(G, with_labels=True)

In [ ]:
best_s, best_d, bes_path, best_cost = None, None, None, float('inf')
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        path = nx.shortest_path(G, s, d, weight='weight')
        cost = cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    if cost < best_cost:
        best_cost = cost
        best_s = s
        best_d = d
        best_path = path
    #ic(s, d, path, cost)
None

In [ ]:
best_s, best_d, best_path, best_cost

In [ ]:
problem[2][4]

In [7]:
import heapq
import numpy as np

def dijkstra(graph, source):
    V = len(graph)
    dist = [float('inf')] * V
    parent = [-1] * V
    dist[source] = 0
    pq = [(0, source)]

    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]:
            continue
        for v, w in graph[u]:
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(pq, (nd, v))

    return dist, parent


def reconstruct_path(parent, src, dst):
    if parent[dst] == -1 and src != dst:
        return None
    path = []
    cur = dst
    while cur != -1:
        path.append(cur)
        if cur == src:
            break
        cur = parent[cur]
    path.reverse()
    return path

def bellman_ford(graph, source):
    V = len(graph)
    dist = [float('inf')] * V
    parent = [-1] * V
    dist[source] = 0

    edges = []
    for u in range(V):
        for v, w in graph[u]:
            edges.append((u, v, w))

    for _ in range(V - 1):
        updated = False
        for u, v, w in edges:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                parent[v] = u
                updated = True
        if not updated:
            break

    # negative cycle
    for u, v, w in edges:
        if dist[u] + w < dist[v]:
            return None, None, True

    return dist, parent, False

def build_graph(problem):
    V = len(problem)
    graph = [[] for _ in range(V)]
    for i in range(V):
        for j in range(V):
            w = problem[i][j]
            if np.isfinite(w):
                graph[i].append((j, float(w)))
    return graph

def landmark_selection(problem, k=30):
    V = len(problem)
    deg = np.sum(np.isfinite(problem), axis=1)
    idx_central = np.argsort(-deg)[:10].tolist()
    idx_periph = np.argsort(deg)[:10].tolist()
    rng = np.random.default_rng(0)
    idx_random = rng.choice(V, size=5, replace=False).tolist()

    finite_means = np.where(
        np.isfinite(problem),
        problem,
        np.nan
    )
    row_mean = np.nanmean(finite_means, axis=1)
    idx_weight_outlier = np.argsort(row_mean)[:5].tolist()
    landmarks = list(dict.fromkeys(
        idx_central + idx_periph + idx_random + idx_weight_outlier
    ))

    return landmarks[:k]


def johnson(problem):
    V = len(problem)
    graph = build_graph(problem)

    # super-source
    graph_ss = graph + [[]]
    for i in range(V):
        graph_ss[V].append((i, 0.0))

    h, parent, neg = bellman_ford(graph_ss, V)
    if neg:
        return None, None, True

    # reweight
    new_graph = [[] for _ in range(V)]
    for u in range(V):
        for v, w in graph[u]:
            new_graph[u].append((v, w + h[u] - h[v]))

    all_dist = []
    all_parent = []

    for u in range(V):
        dist, par = dijkstra(new_graph, u)
        dist_original = [dist[v] - h[u] + h[v] for v in range(V)]
        all_dist.append(dist_original)
        all_parent.append(par)

    return all_dist, all_parent, False


def shortest_path(problem):
    V = len(problem)
    graph = build_graph(problem)

    # --- 1. Negative cycle detection ---
    graph_ss = graph + [[]]
    for i in range(V):
        graph_ss[V].append((i, 0.0))

    _, _, neg = bellman_ford(graph_ss, V)
    if neg:
        return float('-inf'), None

    # --- 2. Complete Johnson for graphs ≤150 ---
    if V <= 150:
        all_dist, all_parent, neg2 = johnson(problem)
        if neg2:
            return float('-inf'), None

        best = float('inf')
        best_pair = None
        best_path = None

        for u in range(V):
            for v in range(V):
                if u == v:
                    continue
                d = all_dist[u][v]
                if d < best:
                    best = d
                    best_pair = (u, v)
                    parent = all_parent[u]
                    best_path = reconstruct_path(parent, u, v)

        return best, (best_pair[0], best_pair[1], best_path)

    # --- 3. Landmark selection for large graphs ---
    landmarks = landmark_selection(problem, k=30)

    best = float('inf')
    best_pair = None
    best_path = None

    for lm in landmarks:
        dist, parent = dijkstra(graph, lm)
        for v in range(V):
            if v == lm:
                continue
            if dist[v] < best:
                best = dist[v]
                best_pair = (lm, v)
                best_path = reconstruct_path(parent, lm, v)

    return best, (best_pair[0], best_pair[1], best_path)


In [10]:
dist, info = shortest_path(problem)

if dist == float('-inf') or info is None:
    print("Negative cycle detected.")
else:
    u, v, path = info
    print("Shortest global distance:", dist)
    print("path:", path)

Shortest global distance: 32.0
path: [909, 940]


In [11]:
for s in sizes:
    for d in density:
        for n in noise_levels:
            for neg in negative_values:
                problem = create_problem(
                    s,
                    density=d,
                    noise_level=n,
                    negative_values=neg
                )
                dist, info = shortest_path(problem)
                if dist == float('-inf') or info is None:
                    print(f"Size: {s}, Density: {d}, Noise: {n}, Negative: {neg} => Negative cycle detected.")
                else:
                    u, v, path = info
                    print(f"Size: {s}, Density: {d}, Noise: {n}, Negative: {neg} => Shortest distance: {dist}, Path length: {len(path)}")

Size: 10, Density: 0.2, Noise: 0.0, Negative: True => Shortest distance: 73.0, Path length: 2
Size: 10, Density: 0.2, Noise: 0.0, Negative: False => Shortest distance: 73.0, Path length: 2
Size: 10, Density: 0.2, Noise: 0.1, Negative: True => Shortest distance: 107.0, Path length: 2
Size: 10, Density: 0.2, Noise: 0.1, Negative: False => Shortest distance: 140.0, Path length: 2
Size: 10, Density: 0.2, Noise: 0.5, Negative: True => Shortest distance: -248.0, Path length: 3
Size: 10, Density: 0.2, Noise: 0.5, Negative: False => Shortest distance: 257.0, Path length: 2
Size: 10, Density: 0.2, Noise: 0.8, Negative: True => Shortest distance: -751.0, Path length: 3
Size: 10, Density: 0.2, Noise: 0.8, Negative: False => Shortest distance: 292.0, Path length: 2
Size: 10, Density: 0.5, Noise: 0.0, Negative: True => Shortest distance: 73.0, Path length: 2
Size: 10, Density: 0.5, Noise: 0.0, Negative: False => Shortest distance: 73.0, Path length: 2
Size: 10, Density: 0.5, Noise: 0.1, Negative: T